## Import Libraries and read data

In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
df = pd.read_csv('C:\data\python\Recommendation-Cross Sales Org.csv')

## Replace N/A with 0

In [39]:
df.fillna(0,inplace=True)
df.head()

,centre_no,qualification_programme_no,2015,2016
0,001025,8600-11,790.0,304.0
1,001025,8600-21,1650.0,214.0
2,001025,8600-22,0.0,255.0
3,001026,E07G-02,1449.0,759.0
4,001036,1016-02,1152.0,1269.0


## Convert % into rate

In [41]:
df.loc[df['2015']==0,'%'] = df['2016']
df.loc[df['2015']!=0,'%'] = df['2016']/df['2015']
min = -0.25
for i in range(10):
    min = min + 0.2
    max = min + 0.2
    if i == 9:
        df.loc[(df['%']>=min),'rate'] = i + 1
    else:
        df.loc[(df['%']>=min)&(df['%']<max),'rate'] = i + 1
df['rate'] = df['rate'].astype(int)
df.count()

centre_no                     82712
qualification_programme_no    82712
2015                          82712
2016                          82712
%                             82712
rate                          82712
dtype: int64

## Remove data products with less than 10 centres

In [70]:
min_no_ctr = 10
centre_qual_number = df.centre_no.value_counts()
df_main = df[df["centre_no"].isin(centre_qual_number[centre_qual_number>min_no_ctr].index)]
df_main.shape

(62043, 6)

## Remove data centres with less than 10 products

In [71]:
min_no_qual = 10
qual_centre_number = df.qualification_programme_no.value_counts()
df_main = df_main[df_main["qualification_programme_no"].isin(qual_centre_number[qual_centre_number>min_no_qual].index)]
df_main.shape

(55738, 6)

In [77]:
df_main.head()

,centre_no,qualification_programme_no,2015,2016,%,rate
4,001036,1016-02,1152.0,1269.0,1.101562,6
5,001036,3296-02,403.0,614.0,1.523573,8
6,001036,3605-03,764.0,492.0,0.643979,4
7,001036,3748-01,650.0,1264.0,1.944615,10
8,001036,3748-02,622.0,1940.0,3.118971,10


## Pivot table : Colum = Products, Row = Centre

In [79]:
userRatings = df_main.pivot_table(index=['centre_no'],columns=['qualification_programme_no'],values='rate')
userRatings.head() 

qualification_programme_no,0059-21,0059-23,0059-31,0065-11,0065-21,0065-23,0065-24,0065-33,0065-34,0065-35,...,TS017131,TS017133,TS027132,TS123008,TS133008,UG/L3,UG/L5,UHC/26,VSEB3605-LV3,WB3/DIG/PL1
centre_no,,,,,,,,,,,,,,,,,,,,,
001036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
001147G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
001227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001229X,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Number of centres & products

In [80]:
userRatings.shape

(1582, 1294)

## Calculate similarity -> Correlations
#### Products correlations
![title](img/corr.png)

In [81]:
corrMatrix = userRatings.corr(method='pearson', min_periods=10)
corrMatrix.head()

qualification_programme_no,0059-21,0059-23,0059-31,0065-11,0065-21,0065-23,0065-24,0065-33,0065-34,0065-35,...,TS017131,TS017133,TS027132,TS123008,TS133008,UG/L3,UG/L5,UHC/26,VSEB3605-LV3,WB3/DIG/PL1
qualification_programme_no,,,,,,,,,,,,,,,,,,,,,
0059-21,1.000000,0.245315,0.440571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0059-23,0.245315,1.000000,0.333094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0059-31,0.440571,0.333094,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0065-11,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0065-21,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.670242,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Centre example : 001036
#### 5 rating lists of the centre
#### '1016-02' -> rating 6

In [133]:
myRatings = userRatings.loc['001036'].dropna()
myRatings.head()

qualification_programme_no
1016-02     6.0
3296-02     8.0
3605-03     4.0
3748-01    10.0
3748-02    10.0
Name: 001036, dtype: float64

##  Product 1016-02
#### correlations of 1016-02 with other products - sort

In [134]:
sims = corrMatrix["1016-02"].dropna()
#sims = corrMatrix[myRatings.index[0]].dropna()
sims.sort_values(inplace = True, ascending = False)
sims.head()

qualification_programme_no
1016-02    1.000000
3800-40    0.907684
3800-50    0.907684
3800-60    0.819369
0070-25    0.817472
Name: 1016-02, dtype: float64

## Calculation: sum ( Correlation * Existing ratings ) for all products 
#### And remove existing products from lists 

In [135]:
simCandidates = pd.Series()
for i in range(0, len(myRatings.index)):
    #print ("Adding sims for " + myRatings.index[i] + "...")
    # Retrieve similar movies to this one that I rated
    sims = corrMatrix[myRatings.index[i]].dropna()
    # Now scale its similarity by how well I rated this movie
    sims = sims.map(lambda x: x * myRatings[i])
    # Add the score to the list of similarity candidates
    simCandidates = simCandidates.append(sims)
    
#Glance at our results so far:
#print ("sorting...")
simCandidates = simCandidates.groupby(simCandidates.index).sum()
simCandidates.sort_values(inplace = True, ascending = False)
filteredSims = simCandidates.drop(myRatings.index,errors='ignore')
print (filteredSims.head(10))

7574-03    57.793427
7132-05    54.327103
3748-03    53.551101
7132-08    51.544955
7384-21    48.174567
4411-33    43.284963
5530-03    43.121877
4340-31    43.066630
6317-32    42.751552
6570-02    42.593454
dtype: float64


## Run for all centres

In [136]:
result_all = pd.DataFrame()
for c in range(0,len(userRatings)):
#for c in range(0,30):
    myRatings = userRatings.iloc[c].dropna()
    simCandidates = pd.Series()
    for i in range(0, len(myRatings.index)):
    #print ("Adding sims for " + myRatings.index[i] + "...")
    # Retrieve similar movies to this one that I rated
        sims = corrMatrix[myRatings.index[i]].dropna()
    # Now scale its similarity by how well I rated this movie
        sims = sims.map(lambda x: x * myRatings[i])
    # Add the score to the list of similarity candidates
        simCandidates = simCandidates.append(sims)
#Glance at our results so far:
#print ("sorting...")
#simCandidates.sort_values(inplace = True, ascending = False)
#print (simCandidates.head(10))

    simCandidates = simCandidates.groupby(simCandidates.index).sum()
    simCandidates.sort_values(inplace = True, ascending = False)
    filteredSims = simCandidates.drop(myRatings.index,errors='ignore')
    filteredSims = filteredSims.head(20)

#pd.DataFrame({'Qual':filteredSims.index, 'Values':filteredSims.values}).transpose()
    result = pd.DataFrame({userRatings.index[c]:filteredSims.index}).transpose()
#userRatings.index[0]
#filteredSims.index
    result_all = result_all.append(result)

In [137]:
result_all.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
001036,7574-03,7132-05,3748-03,7132-08,7384-21,4411-33,5530-03,4340-31,6317-32,6570-02,9873-03,4835-03,5528-02,4835-02,7091-33,5938-01,7108-12,3569-04,9735-02,4227-04
001147,7517-02,3003-47,9735-04,3565-21,4835-02,3003-23,9735-02,4340-31,0071-34,7139-10,4428-01,8625-31,3868-03,9738-04,0748-20,2357-23,3268-13,4227-04,3768-01,7657-21
001147G,9650-03,5546-05,4978-56,7384-21,8623-41,5329-21,4835-03,5528-04,7100-86,4223-11,3003-80,4340-31,6572-01,7734-02,6707-50,2850-20,8610-31,9645-03,5530-03,5938-01
001227,7574-02,9738-02,9738-04,5530-03,3296-02,4227-01,3605-03,4340-31,4233-20,4978-55,4222-31,9865-34,4229-03,9735-02,9873-03,7540-12,4520-03,1016-07,6317-32,6008-02
001229X,6008-05,3605-03,8620-31,4233-20,8621-31,4227-01,4239-02,4223-11,7540-12,6317-33,4270-12,7384-22,4229-03,7108-12,4520-03,TL036008-01,0067-21,4222-31,7574-03,3003-80


In [30]:
result_all.to_csv('C:\\data\\python\\Item_CF01.csv',sep=',')